In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

There was a problem when trying to write in your cache folder (/home/huggingface_cache/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


---

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [4]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 평가모드로 전환
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # torch.no_grad()를 사용하여 테스트 시 변화도(gradient)를 계산하지 않도록 함
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309263  [   64/60000]
loss: 2.291770  [ 6464/60000]
loss: 2.274077  [12864/60000]
loss: 2.262814  [19264/60000]
loss: 2.239475  [25664/60000]
loss: 2.211142  [32064/60000]
loss: 2.223547  [38464/60000]
loss: 2.190748  [44864/60000]
loss: 2.186502  [51264/60000]
loss: 2.134651  [57664/60000]
Test Error: 
 Accuracy: 39.5%, Avg loss: 2.139878 

Epoch 2
-------------------------------
loss: 2.153725  [   64/60000]
loss: 2.138159  [ 6464/60000]
loss: 2.079644  [12864/60000]
loss: 2.091906  [19264/60000]
loss: 2.043247  [25664/60000]
loss: 1.978859  [32064/60000]
loss: 2.013066  [38464/60000]
loss: 1.935220  [44864/60000]
loss: 1.942531  [51264/60000]
loss: 1.849276  [57664/60000]
Test Error: 
 Accuracy: 58.2%, Avg loss: 1.860354 

Epoch 3
-------------------------------
loss: 1.897218  [   64/60000]
loss: 1.859248  [ 6464/60000]
loss: 1.744571  [12864/60000]
loss: 1.778747  [19264/60000]
loss: 1.679679  [25664/60000]
loss: 1.630930  [32064/600